# Modeling Op Amps {#sec-Modeling_Op_Amps}
![Op Amp schematic symbol](ideal-op-amp.png){#fig-ideal-op-amp width=300}  

<!--
## Abstract {.unnumbered}
This notebook examines the Op Amp model used in the Symbolic Modified Analysis python code. Op Amps are a very common componet of electronic circuits and are used extensively in analog signal processing and instrumentation. The Op Amp, within the context of MNA, is modeled as an ideal amplifier with infinite gain and bandwidth. The model assums that circuit surrounding the Op Amp provide a feedback connection from the output termial, $V_{out}$ to the $V_{-}$ input terminal.  
-->

## Introduction
The [Operational Amplifier](https://en.wikipedia.org/wiki/Operational_amplifier) or Op Amp, is type of electronic amplifier that has two input terminals, amplifies the voltage difference between the two input terminals while suppresses any voltage common to the two inputs and is usually operated with negative feedback to control the gain of the amplifier. The amplifier typically has a very high differential mode gain, high input impedance and low output impedance. Today Op Amps are commonly impemnted in an intergrated circuit that can contain hundreds of transistors. The Op Amp is included in my sysmbolic MNA code as one of the element types along with resistors, capacitors, inductors, dependent sources and independednt sources. 

The schematic symbol for an Op Amp is shown in @fig-ideal-op-amp. The two input terminals are labled $V_{-}$ and $V_{+}$. The $V_{-}$ terminal is called the inverting input and $V_{+}$ is called the non-inverting input. $V_{out}$ is the output terminal of the amplifier. An input on the $V_{+}$ terminal causes an in phase output and an input on the $V_{-}$ terminal causes an out of phase output. Not shown, but implied, are power supply and or ground connections. The datasheet for the particular model of Op Amp will show other connections.

Op Amps are very versitile and are used in many types of circuits. A web search will uncover many examples and a few are listed below:

- [Operational Amplifier Applications](https://en.wikipedia.org/wiki/Operational_amplifier_applications#Summing_amplifier)
- [Amplifier Circuit Collection](https://www.ti.com/lit/an/snla140d/snla140d.pdf)
- [Collection of Amp Applications](https://www.analog.com/media/en/technical-documentation/application-notes/28080533an106.pdf)
- [Handbook of Operational Amplifier Applications](https://www.ti.com/lit/an/sboa092b/sboa092b.pdf)
- [Op Amps for Everyone, Ron Mancini](https://web.mit.edu/6.101/www/reference/op_amps_everyone.pdf)

The first mass-produced Op Amp, in 1953, was the vacuum tube-based K2-W from [GAP/R](https://www.sciencedirect.com/topics/engineering/philbrick). The first widely available integrated circuit Op Amp was the Fairchild μA709 operational amplifier, designed by [Bob Widlar](https://en.wikipedia.org/wiki/Bob_Widlar) in 1965. In 1968, the μA709 was replaced by the μA741, which offered better performance while being more stable and easier to setup. A modern version of the μA741 is produced by Texas Instruments, with the part number [LM741](https://www.ti.com/product/LM741). 

There are more than 30 semiconductor manufactures making general purpose Op Amps priced as low as \$0.30 each in small quantities. Under the general purpose catagory, there are thousands of part numbers avaliable from distributers such as [Digikey](https://www.digikey.com/en/products/filter/instrumentation-op-amps-buffer-amps/687?s=N4IgjCBcpgbB0QGMoDMCGAbAzgUwDQgD2UA2iAMwCsADACwAcFIAuoQA4AuUIAypwCcAlgDsA5iAC%2BhMAE4GsqKBSQMOAsTIgaraSAC0AJiXIoggK4aSkclV17jN4uwAE6ALbtWhO4i%2BQwDn9DKklJIA). The world wide market for Op Amps is about \$6 billion dollars.  

Op Amps can be configured to operate in circuits without feedback, which is how a [comparitor](https://en.wikipedia.org/wiki/Comparator) works. Some applications configure the Op Amp to have positive feedback and [Schmitt trigger](https://en.wikipedia.org/wiki/Schmitt_trigger) is an example of this type of configureation. When the Op Amp is used without feedback or with positive feedback, the amplifier is operating as a non-linear device and non-linear circuits cannot be analyized with the Python MNA code presented in this book. 

Only negative feedback Op Amp circuits are supported by the Python MNA code. These circuit configurations typically use resistors to connect the output terminal of the Op Amp to the inverting terminal of the Op Amp. When Op Amps are included in circuits to be analyzed using MNA, the Op Amp model used has negative feed back implied, reguardless of whether such feedback is actually present in the circuit. This is discussed below.  

## The ideal Op Amp
The concept of an ideal Op Amp is used when performance limitations of real Op Amps are ignored in order to symplify the circuit analysis. Often the performance limitions of real Op Amps have only minor effects on the operation of the circuit and the engineer will use their judgment to decide if the second order effects of these limitions need to be considered. The ideal Op Amp is described by @eq-ideal-op-amp. The Op Amp's inputs consist of a non-inverting input $V_+$ and an inverting input $V_-$, as shown in @fig-ideal-op-amp. The Op Amp amplifies only the difference in voltage between the two inputs, which is called the differential input voltage. The power supply terminals of a real Op Amp are not included in the model of an ideal Op Amp. 

The output voltage of the Op Amp $V_{out}$ is given by the equation:

$$V_{out} = A_{OL}(V_+ - V_-)$$ {#eq-ideal-op-amp}

Where $A_{OL}$ is the open-loop gain of the amplifier. The value of the open loop gain in real Op Amps is very large, sometimes on the order of 100 dB. The value of $A_{OL}$ in ideal Op Amps is not needed when the Op Amp is configured with negative feedback. The components in the feedback path determine the gain of the Op Amp circuit independednt of $A_{OL}$.

The ideal Op Amp has other charasterics such as:

- Infinite gain and bandwidth
- Zero output impedance
- Infinite input impedance
- Zero volts between the two inputs
- Infinite output voltage range
- Infinite bandwidth with zero phase shift and infinite slew rate
- Zero noise
- Infinite common-mode rejection ratio

## MNA Op Amp stamp
MNA is an algorithic procedure for generating network equations. When there is an element type 'O', in the netlist, three connection nodes are required. The first node is the inverting input, the second node is the non inverting input and the third node is the output node. The Python code, then makes entries into the B and C matricies based simply on the node numbers of the Op Amp terminals. This is called stamping the matrix, because for each component type a specific template is used. 

The B matrix is an n by m matrix with only 0, 1 and -1 elements, where n = number of nodes and m is the number of current unknowns, i_unk. There is one column for each unknown current. The Op Amp model has one unknown current for the output current. In the code that builds the B matrix, a value of 1 is included in the matrix at the index position given by the Op Amp output node number. The snippit of code that performes this operation is shown below.

```python
# generate the B Matrix
sn = 0   # count source number as code walks through the data frame
for i in range(len(df)):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    n_vout = df.loc[i,'Vout'] # node connected to op amp output

    # process elements with input to B matrix
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    :
    :
    if x == 'O':  # op amp type, output connection of the opamp goes in the B matrix
        B[n_vout-1,sn] = 1
        sn += 1   # increment source count
    :
    :
```
Each Op Amp the entry in the B matrix lines up with an unknown current in the X vector.

The C matrix is an m by n matrix with only 0, 1 and -1 elements. The code is similar to the B matrix code, except the indices are swapped. The code loops through all the branches and for Op Amps, a 1 is inserted for the non-inverting node and a -1 for the inverting node, unless the terminal is connected to ground. The code snippit is shown below.

```python
# generate the C Matrix
sn = 0   # count source number as code walks through the data frame
for i in range(len(df)):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    cn1 = df.loc[i,'cp node'] # nodes for controlled sources
    cn2 = df.loc[i,'cn node']
    n_vout = df.loc[i,'Vout'] # node connected to op amp output

    # process elements with input to B matrix
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    :
    :
    if x == 'O':  # op amp type, input connections of the opamp go into the C matrix
        # C[sn,n_vout-1] = 1
        if i_unk > 1:  #is B greater than 1 by n?, O
            # check to see if any terminal is grounded
            # then stamp the matrix
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1
        sn += 1   # increment source count
    :
    :
```

The above code mathematically equates the voltages at the input input terminals equal to each other, $v_n = v_m$, where n and m are the node numbers. By default, the MNA Op Amp model is operating as if negative feedback is present in the circuit, reguardless of whether such feedback is actually present in the surrounding network. The Python MNA code does not verify that the Op Amp has negative feedback applied by the surrounding network. Consequencely, the Python MNA code is agnostic with reguard to feedback and which input terminal is used. The Python MNA code Op Amp model should be used with some care and the results need to be checked.

The [Nullor](https://en.wikipedia.org/wiki/Nullor) is a two port device which can be used to model ideal Op Amps. The Python NMA Op Amp and the Nullor have the same NMA stamp and are equalivent.  

<!--
Implying that there is negative feedback in the circuit. Doesn't mader which input pin is used for the feedback connection.   

#### Op Amp elements
The op amp element is assumed to be an ideal op amp and use of this component is valid only when used in circuits with a DC path (a short or a resistor) from the output terminal to the negative input terminal of the op amp. No error checking is provided and if the condition is violated, the results likely will be erroneous.   

$\large{\color {red} {\text{Need to fix the references and clean up the description of Op Amps}}}$   
References use in the debugging of the opamp stamp:   

1. Design of Analog Circuits Through Symbolic Analysis, edited by Mourad Fakhfakh, Esteban Tlelo-Cuautle, Francisco V. Fernández   
2. Computer Aided Design and Design Automation, edited by Wai-Kai Chen 

Lines up with an unknown voltages in the X vector

How do the B and C matrices get used?

The A matrix describes the connectivity of the resistors, capacitors and G type (VCCS) circuit elements. The column vector X are the unknown node voltages and unknown currents terms from the voltage sources and inductors. The column vector Z is made of the known voltages and currents. The A is formed by four sub matrices, G, B, C and D, which are described below.

$A = \begin{bmatrix}G B\\C D\end{bmatrix}$

The matrix G is formed from the coefficients representing the KCL equations for each node.

The positive diagonal of $G_{k,k}$ are the conductance terms of the resistor and capacitor elements connected to node k.  The off diagonal terms of $G_{k,j}$ are the resistors and capacitor conductances connecting node k to node j.  G type elements (VCCS) have input to the G matrix at the connection and controlling node positions.

The B matrix describes the connectivity of the unknown branch currents.  Independent voltage sources, opamps, H, F and E type elements as well as inductors have inputs to the B matrix.

The C matrix describes the connectivity of the unknown branch currents and is mainly the transpose of B the matrix, with the exception of the F type elements (CCCS) and includes the E type value. 

The D matrix also describes connectivity of the unknown currents.  The D matrix is composed of zeros unless there are controlled sources and inductors in the network.  

The X vector is composed of the V and J vectors as shown below.   
$X = \begin{bmatrix}V\\J\end{bmatrix}$  
The V vector contains the node voltages which are the voltage unknowns to be solved for.  The J vector contains the unknown currents from each voltage source.

The Z vector is composed of the I and Ev vectors as shown below.  
$Z = \begin{bmatrix}I\\Ev\end{bmatrix}$  
The I vector contains the known currents and the Ev vector contains the known voltages. Ev is used as the variable because sympy uses e and E sometimes for the constant e=2.71, sometimes called Euler's number [[9]](#ref9). The use of E or e as a symbol was causing some errors when the code was run.  

and what does the Op Amp stamp do in the algorithum?

The MNA Op Amp model 

assums that there will be a circuit path from out put to input. This make the circuit linear across all frequencies. The going in assumption with MNA is a linear circuit. No current flow in the input terminals. Where is that defined?

with or with out feed back path, B and C matrices get formulated the same, The difference is in the A matrix. Now there is a connection between Op Amp input and Op Amp output.  With resistive DC path between input and output, there is a proper number of knowns and unknowns.

Checking with C in feedback path

should not have direct L in the feed back path
-->

## Op Amp feedback test circuit
The circuit in @fig-op-amp-test-circuit is used to examine various Op Amp feedback connections. The following Op Amp circuits with various elements in the feedback path are examined below.

1. No feedback
2. R in the feed back path
3. Swap Op Amp input connections
4. C in feedback path
5. C and R in parallel in feedback path  
5a. C and R in series in feedback path
6. L in feedback path
7. L and R in series in the feedback path
8. L and R in parallel in the feedback path

The following Python modules are used in this notebook.

In [1]:
from sympy import *
import numpy as np
from tabulate import tabulate
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()

## Op Amp test circuit
An Op Amp test circuit, shown below, is used to explore various feedback configurations. A resistor is used to terminate each Op Amp terminal. A DC voltage source, V1, is used as the stimulus to the circuit and all the resistor values are set to one.

![Op Amp test circuit](op-amp-test-v1.png){#fig-op-amp-test-circuit}  

## Case 1: No feedback
The net list for @fig-op-amp-test-circuit was obtained from LTSpice.

In [2]:
net_list1 = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
'''

Generate the network equations.

In [3]:
report, network_df, df2, A1, X1, Z1 = SymMNA.smna(net_list1)

# Put matricies into SymPy 
X1 = Matrix(X1)
Z1 = Matrix(Z1)

NE1_sym = Eq(A1*X1,Z1)

Generate markdown text to display the network equations.

In [4]:
temp = ''
for i in range(len(X1)):
    temp += '${:s}$<br>'.format(latex(Eq((A1*X1)[i:i+1][0],Z1[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{O1} + \frac{v_{2}}{R_{3}} = 0$<br>$- \frac{v_{1}}{R_{1}} + \frac{v_{3}}{R_{1}} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>

Shown above are the six equations generatated by NMA. The last three equations are rather simple. 

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [5]:
# turn the free symbols into SymPy variables
var(str(NE1_sym.free_symbols).replace('{','').replace('}',''))

Attemp to solve the equations.

In [6]:
U1_sym = solve(NE1_sym,X1)
U1_sym

No solution is found. By inspection we can see that there are three equations:

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{O1} + \frac{v_{2}}{R_{3}} = 0$<br>$- \frac{v_{1}}{R_{1}} + \frac{v_{3}}{R_{1}} = 0$

and four knowns: $v_2$, $v_3$, $I_{V1}$ and $I_{O1}$.

So without a connection from node 2 to the rest of the network, the voltage and current at node 2 cannot be solved for. 

## Case 2: R in the feed back path
R added to network between nodes 2 and 3, by manually editing the netlist below. 

In [7]:
net_list2 = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
R 2 3 1
'''

Generate the network equations.

In [8]:
report, network_df, df2, A2, X2, Z2 = SymMNA.smna(net_list2)

# Put matricies into SymPy 
X2 = Matrix(X2)
Z2 = Matrix(Z2)

NE2_sym = Eq(A2*X2,Z2)

Both case 1 and case 2 have a 6 by 6 A matrix, but R shows up in the off diagonal in case 2. 

In [9]:
A2

⎡1             -1             ⎤
⎢──     0      ───    0   1  0⎥
⎢R₁             R₁            ⎥
⎢                             ⎥
⎢     1    1   -1             ⎥
⎢ 0   ── + ─   ───    0   0  1⎥
⎢     R₃   R    R             ⎥
⎢                             ⎥
⎢-1    -1     1    1          ⎥
⎢───   ───    ── + ─  0   0  0⎥
⎢ R₁    R     R₁   R          ⎥
⎢                             ⎥
⎢                     1       ⎥
⎢ 0     0       0     ──  0  0⎥
⎢                     R₂      ⎥
⎢                             ⎥
⎢ 1     0       0     0   0  0⎥
⎢                             ⎥
⎣ 0     0       1     -1  0  0⎦

Generate markdown text to display the network equations.

In [10]:
temp = ''
for i in range(len(X2)):
    temp += '${:s}$<br>'.format(latex(Eq((A2*X2)[i:i+1][0],Z2[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{O1} + v_{2} \cdot \left(\frac{1}{R_{3}} + \frac{1}{R}\right) - \frac{v_{3}}{R} = 0$<br>$v_{3} \cdot \left(\frac{1}{R_{1}} + \frac{1}{R}\right) - \frac{v_{1}}{R_{1}} - \frac{v_{2}}{R} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [11]:
# turn the free symbols into SymPy variables
var(str(NE2_sym.free_symbols).replace('{','').replace('}',''))

Solve the equations and display the results.

In [12]:
U2_sym = solve(NE2_sym,X2)

temp = ''
for i in U2_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U2_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = - \frac{R V_{1}}{R_{1}}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{R V_{1} + R_{3} V_{1}}{R_{1} R_{3}}$<br>

Notice that $v_3$ and $v_4$ are both zero volts. There is no current flowing into the Op Amp input terminals.

Solving for the transfer function at node 2.

In [13]:
H2_sym = U2_sym[v2]/U2_sym[v1]
H2_sym #.simplify()

## Case 3: Swap Op Amp input connections
As discussed above, the MNA code equates the voltages at the Op Amps input terminals. This implies that there is negative feedback present in the circuit. Because the voltage at thes terminal are equal, there is no distinction between the two as far as a mathematical solution to the network equations is concerned. This is illustrated below.

In [14]:
net_list3 = '''
V1 1 0 1
*O1 3 4 2
O1 4 3 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
R 2 3 1
'''

Generate the network equations.

In [15]:
report, network_df, df2, A3, X3, Z3 = SymMNA.smna(net_list3)

# Put matricies into SymPy 
X3 = Matrix(X3)
Z3 = Matrix(Z3)

NE3_sym = Eq(A3*X3,Z3)

Generate markdown text to display the network equations.

In [16]:
temp = ''
for i in range(len(X3)):
    temp += '${:s}$<br>'.format(latex(Eq((A3*X3)[i:i+1][0],Z3[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{O1} + v_{2} \cdot \left(\frac{1}{R_{3}} + \frac{1}{R}\right) - \frac{v_{3}}{R} = 0$<br>$v_{3} \cdot \left(\frac{1}{R_{1}} + \frac{1}{R}\right) - \frac{v_{1}}{R_{1}} - \frac{v_{2}}{R} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$- v_{3} + v_{4} = 0$<br>

Last equation,

$- v_{3} + v_{4} = 0$

is identical anlgebracily to the equation generated in Case 1 above, which is:

$v_{3} - v_{4} = 0$

This shows that because the Op Amp model defines $V_+$ and $V_-$ to be equal, the terminals of the NMA Op Amp model can be swapped.

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [17]:
# turn the free symbols into SymPy variables
var(str(NE3_sym.free_symbols).replace('{','').replace('}',''))

Solve the equations and display the results.

In [18]:
U3_sym = solve(NE3_sym,X3)

temp = ''
for i in U3_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U3_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = - \frac{R V_{1}}{R_{1}}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{R V_{1} + R_{3} V_{1}}{R_{1} R_{3}}$<br>

Notice that $v_3$ and $v_4$ are both zero volts. There is no current flowing into the Op Amp input terminals.

Solving for the transfer function at node 2.

In [19]:
H3_sym = U3_sym[v2]/U3_sym[v1]
H3_sym #.simplify()

## Case 4: C in feedback path

In [20]:
net_list4 = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
C 2 3 1
'''

Generate the network equations.

In [21]:
report, network_df, df2, A4, X4, Z4 = SymMNA.smna(net_list4)

# Put matricies into SymPy 
X4 = Matrix(X4)
Z4 = Matrix(Z4)

NE4_sym = Eq(A4*X4,Z4)

Generate markdown text to display the network equations.

In [22]:
temp = ''
for i in range(len(X4)):
    temp += '${:s}$<br>'.format(latex(Eq((A4*X4)[i:i+1][0],Z4[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$- C s v_{3} + I_{O1} + v_{2} \left(C s + \frac{1}{R_{3}}\right) = 0$<br>$- C s v_{2} + v_{3} \left(C s + \frac{1}{R_{1}}\right) - \frac{v_{1}}{R_{1}} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [23]:
# turn the free symbols into SymPy variables
var(str(NE4_sym.free_symbols).replace('{','').replace('}',''))

Solve and display the solution.

In [24]:
U4_sym = solve(NE4_sym,X4)

temp = ''
for i in U4_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U4_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = - \frac{V_{1}}{C R_{1} s}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{C R_{3} V_{1} s + V_{1}}{C R_{1} R_{3} s}$<br>

Solving for the transfer function at node 2.

In [25]:
U4_sym[v2]/U4_sym[v1]

Seems correct.

## Case 5: C and R in parallel in feedback path

In [26]:
net_list5 = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
C 2 3 1
R 2 3 1
'''

Generate the network equations.

In [27]:
report, network_df, df2, A5, X5, Z5 = SymMNA.smna(net_list5)

# Put matricies into SymPy 
X5 = Matrix(X5)
Z5 = Matrix(Z5)

NE5_sym = Eq(A5*X5,Z5)

Generate markdown text to display the network equations.

In [28]:
temp = ''
for i in range(len(X5)):
    temp += '${:s}$<br>'.format(latex(Eq((A5*X5)[i:i+1][0],Z5[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{O1} + v_{2} \left(C s + \frac{1}{R_{3}} + \frac{1}{R}\right) + v_{3} \left(- C s - \frac{1}{R}\right) = 0$<br>$v_{2} \left(- C s - \frac{1}{R}\right) + v_{3} \left(C s + \frac{1}{R_{1}} + \frac{1}{R}\right) - \frac{v_{1}}{R_{1}} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [29]:
# turn the free symbols into SymPy variables
var(str(NE5_sym.free_symbols).replace('{','').replace('}',''))

Solve and display the solution.

In [30]:
U5_sym = solve(NE5_sym,X5)

temp = ''
for i in U5_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U5_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = - \frac{R V_{1}}{C R R_{1} s + R_{1}}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{C R R_{3} V_{1} s + R V_{1} + R_{3} V_{1}}{C R R_{1} R_{3} s + R_{1} R_{3}}$<br>

Solving for the transfer function at node 2.

In [31]:
U5_sym[v2]/U5_sym[v1]

At $s=0$ the transfer function reduces correctly to the DC case. For $s=\infty$, the capacitor is a short and equation is not correct, but then the circuit now has a short accross R and this is not reflected in the netlist. So at $s=\infty$, the solution needs some interperation and maybe the netlist needs to edited to correctly reflect the value of $Z_c$ at $\infty$.

The result seems correct for $s \ne \infty$.

## Case 5a: C and R in series in feedback path

In [32]:
net_list5a = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
C 2 5 1
R 3 5 1
'''

Generate the network equations.

In [33]:
report, network_df, df2, A5a, X5a, Z5a = SymMNA.smna(net_list5a)

# Put matricies into SymPy 
X5a = Matrix(X5a)
Z5a = Matrix(Z5a)

NE5a_sym = Eq(A5a*X5a,Z5a)

Generate markdown text to display the network equations.

In [34]:
temp = ''
for i in range(len(X5a)):
    temp += '${:s}$<br>'.format(latex(Eq((A5a*X5a)[i:i+1][0],Z5a[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$- C s v_{5} + I_{O1} + v_{2} \left(C s + \frac{1}{R_{3}}\right) = 0$<br>$v_{3} \cdot \left(\frac{1}{R_{1}} + \frac{1}{R}\right) - \frac{v_{1}}{R_{1}} - \frac{v_{5}}{R} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$- C s v_{2} + v_{5} \left(C s + \frac{1}{R}\right) - \frac{v_{3}}{R} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [35]:
# turn the free symbols into SymPy variables
var(str(NE5a_sym.free_symbols).replace('{','').replace('}',''))

Solve and display the solution.

In [36]:
U5a_sym = solve(NE5a_sym,X5a)

temp = ''
for i in U5a_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U5a_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = \frac{- C R V_{1} s - V_{1}}{C R_{1} s}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$v_{5} = - \frac{R V_{1}}{R_{1}}$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{C R V_{1} s + C R_{3} V_{1} s + V_{1}}{C R_{1} R_{3} s}$<br>

Solving for the transfer function at node 2.

In [37]:
(U5a_sym[v2]/U5a_sym[v1]).simplify()

The result seems correct.

## Case 6: L in feedback path

In [38]:
net_list6 = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
L 2 3 1
'''

Generate the network equations.

In [39]:
report, network_df, df2, A6, X6, Z6 = SymMNA.smna(net_list6)

# Put matricies into SymPy 
X6 = Matrix(X6)
Z6 = Matrix(Z6)

NE6_sym = Eq(A6*X6,Z6)

Generate markdown text to display the network equations.

In [40]:
temp = ''
for i in range(len(X6)):
    temp += '${:s}$<br>'.format(latex(Eq((A6*X6)[i:i+1][0],Z6[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{L} + I_{O1} + \frac{v_{2}}{R_{3}} = 0$<br>$- I_{L} - \frac{v_{1}}{R_{1}} + \frac{v_{3}}{R_{1}} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>$- I_{L} L s + v_{2} - v_{3} = 0$<br>

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [41]:
# turn the free symbols into SymPy variables
var(str(NE6_sym.free_symbols).replace('{','').replace('}',''))

Solve and display the solution.

In [42]:
U6_sym = solve(NE6_sym,X6)

temp = ''
for i in U6_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U6_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = - \frac{L V_{1} s}{R_{1}}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{L V_{1} s + R_{3} V_{1}}{R_{1} R_{3}}$<br>$I_{L} = - \frac{V_{1}}{R_{1}}$<br>

Solving for the transfer function at node 2.

In [43]:
U6_sym[v2]/U6_sym[v1]

At $s=0$ the transfer function does not reduce correctly to the DC case. For $s=\infty$, the inductor is an open and equation is correct.

The result seems correct for $s \ne 0$.

## Case 7: L and R in series in the feedback path

In [44]:
net_list7 = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
L 2 5 1
R 3 5 1
'''

Generate the network equations.

In [45]:
report, network_df, df2, A7, X7, Z7 = SymMNA.smna(net_list7)

# Put matricies into SymPy 
X7 = Matrix(X7)
Z7 = Matrix(Z7)

NE7_sym = Eq(A7*X7,Z7)

Generate markdown text to display the network equations.

In [46]:
temp = ''
for i in range(len(X7)):
    temp += '${:s}$<br>'.format(latex(Eq((A7*X7)[i:i+1][0],Z7[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{L} + I_{O1} + \frac{v_{2}}{R_{3}} = 0$<br>$v_{3} \cdot \left(\frac{1}{R_{1}} + \frac{1}{R}\right) - \frac{v_{1}}{R_{1}} - \frac{v_{5}}{R} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$- I_{L} - \frac{v_{3}}{R} + \frac{v_{5}}{R} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>$- I_{L} L s + v_{2} - v_{5} = 0$<br>

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [47]:
# turn the free symbols into SymPy variables
var(str(NE7_sym.free_symbols).replace('{','').replace('}',''))

Solve and display the solution.

In [48]:
U7_sym = solve(NE7_sym,X7)

temp = ''
for i in U7_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U7_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = \frac{- L V_{1} s - R V_{1}}{R_{1}}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$v_{5} = - \frac{R V_{1}}{R_{1}}$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{L V_{1} s + R V_{1} + R_{3} V_{1}}{R_{1} R_{3}}$<br>$I_{L} = - \frac{V_{1}}{R_{1}}$<br>

Solving for the transfer function at node 2.

In [49]:
(U7_sym[v2]/U7_sym[v1]).simplify()

The result seems correct for $s \ne \infty$.

## Case 8: L and R in parallel in the feedback path

In [50]:
net_list8 = '''
V1 1 0 1
O1 3 4 2
R1 3 1 1
R3 2 0 1
R2 4 0 1
L 2 3 1
R 2 3 1
'''

Generate the network equations.

In [51]:
report, network_df, df2, A8, X8, Z8 = SymMNA.smna(net_list8)

# Put matricies into SymPy 
X8 = Matrix(X8)
Z8 = Matrix(Z8)

NE8_sym = Eq(A8*X8,Z8)

Generate markdown text to display the network equations.

In [52]:
temp = ''
for i in range(len(X8)):
    temp += '${:s}$<br>'.format(latex(Eq((A8*X8)[i:i+1][0],Z8[i])))

Markdown(temp)

$I_{V1} + \frac{v_{1}}{R_{1}} - \frac{v_{3}}{R_{1}} = 0$<br>$I_{L} + I_{O1} + v_{2} \cdot \left(\frac{1}{R_{3}} + \frac{1}{R}\right) - \frac{v_{3}}{R} = 0$<br>$- I_{L} + v_{3} \cdot \left(\frac{1}{R_{1}} + \frac{1}{R}\right) - \frac{v_{1}}{R_{1}} - \frac{v_{2}}{R} = 0$<br>$\frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$v_{3} - v_{4} = 0$<br>$- I_{L} L s + v_{2} - v_{3} = 0$<br>

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [53]:
# turn the free symbols into SymPy variables
var(str(NE8_sym.free_symbols).replace('{','').replace('}',''))

Solve the equations.

In [54]:
U8_sym = solve(NE8_sym,X8)

temp = ''
for i in U8_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U8_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = - \frac{L R V_{1} s}{L R_{1} s + R R_{1}}$<br>$v_{3} = 0$<br>$v_{4} = 0$<br>$I_{V1} = - \frac{V_{1}}{R_{1}}$<br>$I_{O1} = \frac{L R V_{1} s + L R_{3} V_{1} s + R R_{3} V_{1}}{L R_{1} R_{3} s + R R_{1} R_{3}}$<br>$I_{L} = - \frac{R V_{1}}{L R_{1} s + R R_{1}}$<br>

Solving for the transfer function at node 2.

In [55]:
(U8_sym[v2]/U8_sym[v1]).simplify()

The result seems correct for $s \ne \infty$ or $s \ne 0$.

## Summary

- Resistors in the feedback path seem to give the correct results
- Inductors and capacitors in the feedback path seem to work as long as $s \ne \infty \space \text {or} \space \ne 0$. 
- If a solution is only desired for non-zero or non-infinit requencies, then the results seem correct.